In [1]:
pip install tenacity


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt,wait_exponential
import urllib
import time
import numpy as np
import pandas as pd

#tenacityのライブラリに関する記事は下記
#https://zenn.dev/taroman_zenn/articles/dd0b33a3a37d1e

In [3]:
#Step1-1必要な情報を取得する

#SUUMO上で必要な情報を検索する
#今回はペルソナに合致した下記の条件を設定｜ペルソナ：https://www.dropbox.com/scl/fi/pdo0x2sm6es6n76sek2i7/Tech0_STEP3-1_-_v1.0.pdf?rlkey=2attz3z14yyoanwm53spxk6t1&dl=0
#検索条件：東京都内、20万円以下（管理費・共益費込み）、2K以上の間取り、駅徒歩10分以内、バストイレ別、室内洗濯機置場
#取得したURL：https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&ta=13&cb=0.0&ct=20.0&co=1&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=10&mb=0&mt=9999999&cn=9999999&tc=0400301&tc=0400501&fw2=

#複数ページの情報をまとめて取得するリストを作成
data_samples = []

#スクレイピングするページ数を指定
#1ページの表示件数を最大の50件に設定した上で、出力されたページ数237を上回るように設定する
max_page = 240

#SUUMOで自分が調べたい地域を指定して検索し、出力したページのURL
URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&ta=13&cb=0.0&ct=20.0&co=1&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=10&mb=0&mt=9999999&cn=9999999&tc=0400301&tc=0400501&fw2=&page={:d}'
suumo_URL ='https://suumo.jp/'

#試行回数3回、リトライ間の初期待機時間10秒、連続するリトライの時間を100秒にする（指数関数的に増やすが、100秒までで制限する）
#Pythonでは、デコレータの引数を|で結合することはできないため、,で指定すること
@retry(stop=stop_after_attempt(3),wait=wait_exponential(multiplier=10, min=10, max=100))
def load_page(URL):
    html = requests.get(URL)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup


In [4]:
# 処理時間を測りたい
start = time.time()
times = []

#関数については下記を参考にすること
#enumerate()関数 https://note.nkmk.me/python-enumerate-start/
#strip()関数 https://note.nkmk.me/python-str-remove-strip/#strip

for page in range(1,max_page+1):
    before = time.time()
    #ページ情報
    soup = load_page(URL.format(page))
    #物件リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    #物件ごとの処理
    for child in mother:
        
        #建物情報
        data_home = []
        #カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        #建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        #住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        #最寄駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id, grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        #築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)
        
        #部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            #部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                #階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                #家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                #敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                #間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                #url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(suumo_URL, get_url)
                    data_room.append(abs_url)
            #物件情報と部屋情報を結合する
            data_sample = data_home + data_room,
            data_samples.append(data_sample)

    #1アクセスごとに1秒休む
    time.sleep(1)
    
    #進捗確認
    #このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'総取得件数:{len(data_samples)}')
    #作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了:{complete_ratio}%')
    #作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間:{hour}時間{minute}分{second}秒\n')

    
# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

総取得件数:180
完了:0.417%
残り時間:0時間14分36秒

総取得件数:256
完了:0.833%
残り時間:0時間13分47秒

総取得件数:355
完了:1.25%
残り時間:0時間13分1秒

総取得件数:461
完了:1.667%
残り時間:0時間12分48秒

総取得件数:630
完了:2.083%
残り時間:0時間12分48秒

総取得件数:741
完了:2.5%
残り時間:0時間13分7秒

総取得件数:851
完了:2.917%
残り時間:0時間12分47秒

総取得件数:967
完了:3.333%
残り時間:0時間12分50秒

総取得件数:1057
完了:3.75%
残り時間:0時間12分42秒

総取得件数:1131
完了:4.167%
残り時間:0時間12分26秒

総取得件数:1209
完了:4.583%
残り時間:0時間12分14秒

総取得件数:1278
完了:5.0%
残り時間:0時間12分3秒

総取得件数:1368
完了:5.417%
残り時間:0時間11分56秒

総取得件数:1448
完了:5.833%
残り時間:0時間11分52秒

総取得件数:1524
完了:6.25%
残り時間:0時間11分49秒

総取得件数:1612
完了:6.667%
残り時間:0時間11分43秒

総取得件数:1684
完了:7.083%
残り時間:0時間11分35秒

総取得件数:1762
完了:7.5%
残り時間:0時間11分27秒

総取得件数:1860
完了:7.917%
残り時間:0時間11分23秒

総取得件数:1938
完了:8.333%
残り時間:0時間11分15秒

総取得件数:2016
完了:8.75%
残り時間:0時間11分8秒

総取得件数:2090
完了:9.167%
残り時間:0時間11分5秒

総取得件数:2179
完了:9.583%
残り時間:0時間11分0秒

総取得件数:2255
完了:10.0%
残り時間:0時間10分54秒

総取得件数:2329
完了:10.417%
残り時間:0時間10分50秒

総取得件数:2411
完了:10.833%
残り時間:0時間10分46秒

総取得件数:2491
完了:11.25%
残り時間:0時間10分41秒

総取得件数:2563
完了:11.667%
残り

In [47]:
#Step1-2データを整形する

#リスト内包表記を使用し、data_samplesリストの各要素からアイテムを取得し、新しいリスト split_data_samplesへ格納する
#item[0]となっているのは、タプル内の要素が1つのリストのみであるため＝([],)となっている
split_data_samples = [item[0] for item in data_samples]

# DataFrame の作成
df = pd.DataFrame(split_data_samples)

df.columns = ['カテゴリ', '物件名', '所在地', 'アクセス1', 'アクセス2', 'アクセス3', '築年数', '建物の階数', '部屋の階数', '賃料', '管理費', '敷金', '礼金', '間取り', '面積', 'URL']
df

,カテゴリ,物件名,所在地,アクセス1,アクセス2,アクセス3,築年数,建物の階数,部屋の階数,賃料,管理費,敷金,礼金,間取り,面積,URL
0,賃貸アパート,西武新宿線 上石神井駅 2階建 築8年,東京都練馬区上石神井２,西武新宿線/上石神井駅 歩4分,西武新宿線/上井草駅 歩19分,西武新宿線/武蔵関駅 歩20分,築8年,2階建,1-2階,18.5万円,4000円,-,18.5万円,20K,78.96m2,https://suumo.jp/chintai/jnc_000085747306/?bc=...
1,賃貸アパート,西武新宿線 上石神井駅 2階建 築8年,東京都練馬区上石神井２,西武新宿線/上石神井駅 歩4分,西武新宿線/上井草駅 歩19分,西武新宿線/武蔵関駅 歩20分,築8年,2階建,1-2階,18.5万円,4000円,-,18.5万円,3LDK,78.96m2,https://suumo.jp/chintai/jnc_000085810437/?bc=...
2,賃貸一戸建て,東武伊勢崎線 小菅駅 2階建 築34年,東京都足立区西綾瀬１,東武伊勢崎線/小菅駅 歩6分,ＪＲ常磐線/綾瀬駅 歩10分,東京メトロ千代田線/綾瀬駅 歩10分,築34年,2階建,1-2階,14.8万円,-,29.6万円,14.8万円,4LDK,109.61m2,https://suumo.jp/chintai/jnc_000087224284/?bc=...
3,賃貸マンション,アーブル下落合Ａ棟,東京都新宿区下落合２,ＪＲ山手線/高田馬場駅 歩7分,東京メトロ東西線/高田馬場駅 歩7分,ＪＲ山手線/目白駅 歩7分,築12年,4階建,1階,10万円,4000円,10万円,10万円,2K,31.01m2,https://suumo.jp/chintai/jnc_000087358522/?bc=...
4,賃貸一戸建て,ＪＲ京浜東北線 王子駅 2階建 築64年,東京都北区栄町,ＪＲ京浜東北線/王子駅 歩10分,都電荒川線/栄町駅 歩5分,東京メトロ南北線/西ケ原駅 歩13分,築64年,2階建,1-2階,12万円,5000円,12万円,-,4LDK,55.36m2,https://suumo.jp/chintai/jnc_000084344574/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15766,賃貸一戸建て,都営大江戸線 西新宿五丁目駅 2階建 築56年,東京都新宿区西新宿４,都営大江戸線/西新宿五丁目駅 歩2分,東京メトロ丸ノ内線/西新宿駅 歩11分,都営大江戸線/都庁前駅 歩12分,築56年,2階建,1階,15.9万円,10000円,15.9万円,15.9万円,2LDK,49.56m2,https://suumo.jp/chintai/jnc_000086693431/?bc=...
15767,賃貸アパート,木原山ロッヂ,東京都大田区山王４,ＪＲ京浜東北線/大森駅 歩10分,都営浅草線/馬込駅 歩16分,京急本線/大森海岸駅 歩21分,築85年,3階建,1階,12万円,-,12万円,-,2LDK,54.15m2,https://suumo.jp/chintai/jnc_000081290571/?bc=...
15768,賃貸マンション,山王パレス,東京都大田区山王２,ＪＲ京浜東北線/大森駅 歩2分,京急本線/大森海岸駅 歩14分,京急本線/平和島駅 歩26分,築57年,7階建,5階,18万円,-,18万円,18万円,2LDK,53.71m2,https://suumo.jp/chintai/jnc_000083331187/?bc=...
15769,賃貸アパート,グリーンハイム,東京都練馬区関町南３,ＪＲ中央線/吉祥寺駅 バス8分 (バス停)関町南三丁目 歩5分,西武新宿線/武蔵関駅 歩10分,西武新宿線/上石神井駅 歩21分,築31年,2階建,1階,8万円,-,8万円,-,2DK,44.61m2,https://suumo.jp/chintai/jnc_000087119366/?bc=...


In [48]:
#Step1-3
#出力した結果をCSVファイルに保存する
df.to_csv('suumo_data.csv', index=False, encoding='utf-8-sig')

In [3]:
#データクレンジングに必要なライブラリをインポートする
import csv
import re

In [50]:
#関数定義

#データ型を確認するための関数
def check_data(df):
    print(df.dtypes) #各列のデータ型を確認
    print(df.isna().any()) #DataFrame内のNaNが含まれているか確認

#特定のカラムのデータをint型に変換するための関数
def change_int(column):
    df[column] = df[column].astype(int)

#特定カラムのデータのみをcsvに出力するための関数
def export_csv(columns,df_check_name, csv_name): 
    df_check_name = df[columns]
    file_name = f"{csv_name}.csv"  # 出力ファイル名を生成
    df_check_name.to_csv(file_name) # CSVファイルに出力

In [51]:
#csvデータの読み込み
df = pd.read_csv('suumo_data.csv')
df

,カテゴリ,物件名,所在地,アクセス1,アクセス2,アクセス3,築年数,建物の階数,部屋の階数,賃料,管理費,敷金,礼金,間取り,面積,URL
0,賃貸アパート,西武新宿線 上石神井駅 2階建 築8年,東京都練馬区上石神井２,西武新宿線/上石神井駅 歩4分,西武新宿線/上井草駅 歩19分,西武新宿線/武蔵関駅 歩20分,築8年,2階建,1-2階,18.5万円,4000円,-,18.5万円,20K,78.96m2,https://suumo.jp/chintai/jnc_000085747306/?bc=...
1,賃貸アパート,西武新宿線 上石神井駅 2階建 築8年,東京都練馬区上石神井２,西武新宿線/上石神井駅 歩4分,西武新宿線/上井草駅 歩19分,西武新宿線/武蔵関駅 歩20分,築8年,2階建,1-2階,18.5万円,4000円,-,18.5万円,3LDK,78.96m2,https://suumo.jp/chintai/jnc_000085810437/?bc=...
2,賃貸一戸建て,東武伊勢崎線 小菅駅 2階建 築34年,東京都足立区西綾瀬１,東武伊勢崎線/小菅駅 歩6分,ＪＲ常磐線/綾瀬駅 歩10分,東京メトロ千代田線/綾瀬駅 歩10分,築34年,2階建,1-2階,14.8万円,-,29.6万円,14.8万円,4LDK,109.61m2,https://suumo.jp/chintai/jnc_000087224284/?bc=...
3,賃貸マンション,アーブル下落合Ａ棟,東京都新宿区下落合２,ＪＲ山手線/高田馬場駅 歩7分,東京メトロ東西線/高田馬場駅 歩7分,ＪＲ山手線/目白駅 歩7分,築12年,4階建,1階,10万円,4000円,10万円,10万円,2K,31.01m2,https://suumo.jp/chintai/jnc_000087358522/?bc=...
4,賃貸一戸建て,ＪＲ京浜東北線 王子駅 2階建 築64年,東京都北区栄町,ＪＲ京浜東北線/王子駅 歩10分,都電荒川線/栄町駅 歩5分,東京メトロ南北線/西ケ原駅 歩13分,築64年,2階建,1-2階,12万円,5000円,12万円,-,4LDK,55.36m2,https://suumo.jp/chintai/jnc_000084344574/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15766,賃貸一戸建て,都営大江戸線 西新宿五丁目駅 2階建 築56年,東京都新宿区西新宿４,都営大江戸線/西新宿五丁目駅 歩2分,東京メトロ丸ノ内線/西新宿駅 歩11分,都営大江戸線/都庁前駅 歩12分,築56年,2階建,1階,15.9万円,10000円,15.9万円,15.9万円,2LDK,49.56m2,https://suumo.jp/chintai/jnc_000086693431/?bc=...
15767,賃貸アパート,木原山ロッヂ,東京都大田区山王４,ＪＲ京浜東北線/大森駅 歩10分,都営浅草線/馬込駅 歩16分,京急本線/大森海岸駅 歩21分,築85年,3階建,1階,12万円,-,12万円,-,2LDK,54.15m2,https://suumo.jp/chintai/jnc_000081290571/?bc=...
15768,賃貸マンション,山王パレス,東京都大田区山王２,ＪＲ京浜東北線/大森駅 歩2分,京急本線/大森海岸駅 歩14分,京急本線/平和島駅 歩26分,築57年,7階建,5階,18万円,-,18万円,18万円,2LDK,53.71m2,https://suumo.jp/chintai/jnc_000083331187/?bc=...
15769,賃貸アパート,グリーンハイム,東京都練馬区関町南３,ＪＲ中央線/吉祥寺駅 バス8分 (バス停)関町南三丁目 歩5分,西武新宿線/武蔵関駅 歩10分,西武新宿線/上石神井駅 歩21分,築31年,2階建,1階,8万円,-,8万円,-,2DK,44.61m2,https://suumo.jp/chintai/jnc_000087119366/?bc=...


In [52]:
#Step2-1｜アクセスを分割する

##/で区切られている内容を分割する（路線名と駅名に分ける）
def split_access_column(df, access_column, train_column, new_param_1):
    def safe_split(x):
        # '/'で分割し、2つの要素を持つリストを返す
        parts = x.split('/') if pd.notna(x) else [None, None]
        if len(parts) != 2:  # 期待される2つの要素がない場合
            return [None, None]  # 安全な値を返す
        return parts

    # 安全な分割関数を適用
    split_data = df[access_column].apply(safe_split)
    # 分割したデータを2つの新しい列に格納
    df[[train_column, new_param_1]] = pd.DataFrame(split_data.tolist(), index=df.index)
    return df


##駅名と付随している所要時間を分割する関数
def split_station_and_bus_and_walk(df, new_param_1, station_column, new_param_2):
    re_split_data = df[new_param_1].apply(lambda x: x.split(maxsplit=1) if pd.notna(x) else [None, None])
    df[[station_column, new_param_2]] = pd.DataFrame(re_split_data.tolist(), index=df.index)
    return df

##バスと徒歩を抜き出すための関数
def process_bus_and_walk_data(df, new_param_2,bus_min_column,walk_min_column):
    def split_bus_and_walk(text):
        if pd.notna(text):
            bus_match = re.search(r'バス\d+分', text)
            walk_match = re.search(r'歩\d+分', text)
    
            bus_time = bus_match.group(0) if bus_match else ''
            walk_time = walk_match.group(0) if walk_match else ''
    
            if 'バス' in text:
                return bus_time, walk_time
            else:
                return '', walk_time
        else:
            return [None, None]
    df[[bus_min_column, walk_min_column]] = pd.DataFrame(df[new_param_2].apply(split_bus_and_walk).tolist(), index=df.index)
    return df

##バスと徒歩について不要な文字を削除する
def delete_text_bus_and_walk(df, bus_min_column,walk_min_column):
    df[bus_min_column] = df[bus_min_column].str.replace('バス', '').str.replace('分', '')
    df[walk_min_column] = df[walk_min_column].str.replace('歩', '').str.replace('分', '')
    return df

#上記をまとめて実行すための関数
def process_transport_data(df, access_column, train_column,station_column, new_param_1, new_param_2,bus_min_column,walk_min_column):
    # 最初の分割：'/'で路線名と駅名に分割
    df = split_access_column(df, access_column, train_column, new_param_1)
    # 二番目の分割：駅名と所要時間を分割
    df = split_station_and_bus_and_walk(df, new_param_1, station_column, new_param_2)
    # バスと徒歩の所要時間を抽出
    df = process_bus_and_walk_data(df, new_param_2,bus_min_column,walk_min_column)
    df= df.drop(columns=new_param_1) #不要な列を削除する
    df = df.drop(columns=new_param_2) #不要な列を削除する
    df = delete_text_bus_and_walk(df, bus_min_column,walk_min_column)
    return df


df = process_transport_data(df, 'アクセス1', '路線1', '駅名1', 'new_param_1_1', 'new_param_1_2','バス所要時間1','徒歩所要時間1')
df = process_transport_data(df, 'アクセス2', '路線2', '駅名2', 'new_param_2_1', 'new_param_2_2','バス所要時間2','徒歩所要時間2')
df = process_transport_data(df, 'アクセス3', '路線3', '駅名3', 'new_param_3_1', 'new_param_3_2','バス所要時間3','徒歩所要時間3')


In [53]:
#Step2-2｜築年数/建物の階数/部屋の階数のデータ整理を行う

#特定のカラムについて指定した文字を削除し、int系に変換する
def delete_text_int(df, column,delete_texts):
    for delete_text_int in delete_texts:
        df[column] = df[column].str.replace(delete_text_int,'') #指定した文字を削除する
        df[column] = df[column].apply(lambda x: int(x) if pd.notna(x) and x.isdigit() else x)
    return df

df = delete_text_int(df, '築年数',['築','年']) #築年数のカラムから「築」を削除する
df = delete_text_int(df, '建物の階数','階建') #建物の階段のカラムから「階建」を削除する
df = delete_text_int(df, '部屋の階数','階') #部屋の階数のカラムから「階」を削除する

In [54]:
#Step2-3｜賃料/管理費/敷金/礼金のデータ整理を行う

##ハイフンが入っている場合に0を返す関数
def hyphen_to_zero(value):
    if isinstance(value, str) and '-' in value:
        return '0'
    else:
        return value

##各カラムから指定した文字を削除し、int型に修正するための関数
def delete_text_money(df, column,delete_texts):
    df[column] = df[column].apply(hyphen_to_zero) # '-'が含まれる行に0を入力
    df[column] = df[column].apply(b_to_zero) # '-'が含まれる行に0を入力
    df[column] = df[column].astype(str)  # 列を文字列型に変換
    for delete_text in delete_texts:
        df[column] = df[column].str.replace(delete_text, '', regex=False)  # 指定した文字を削除
    df[column] = df[column].astype(float) * 10000
    df[column] = df[column].astype(int)
    return df

df = delete_text_money(df, '賃料', '万円') #賃料のカラムから「万円」を削除する
df = delete_text_money(df, '敷金', '万円') #賃料のカラムから「万円」を削除する
df = delete_text_money(df, '礼金', '万円') #賃料のカラムから「万円」を削除する

df = delete_text_int(df, '管理費','円')  #管理費のカラムから「円」を削除する



In [55]:
#Step2-4｜データ型をint型に変換する（管理費/築年数/建物の階数/部屋の階数）

#管理費を処理する
df['管理費'] = df['管理費'].apply(hyphen_to_zero) # 管理費のカラムについて、'-'が含まれる行に0を入力
change_int('管理費') #管理費をint型に修正する

#築年数の変換
df['築年数'] = df['築年数'].astype(str).str.replace('新', '0')  #築年数のカラムから「新」を削除し、0を投入
df['築年数'] = df['築年数'].astype(str).str.replace('以上', '')  #築年数のカラムから「新」を削除し、0を投入
df['築年数'] = df['築年数'].replace('', '0')  # 空の文字列を'0'に置換
change_int('築年数') #築年数をint型に修正する

#建物の階数の変換
df['建物の階数'] = df['建物の階数'].astype(str).str.replace('平屋', '1')  #建物の階数のカラムから「平屋」を削除し、1を投入
# '地上'に続く数字を抽出する関数
def extract_floors(text):
    if '地上' in text:
        return pd.Series(text).str.extract(r'地上(\d+)',expand=False)[0]
    else:
        return text
df['建物の階数'] = df['建物の階数'].apply(extract_floors) #関数を適用して抽出
change_int('建物の階数') #築年数をint型に修正する

#部屋の階数の変換
df.loc[df['部屋の階数'] == '-', '部屋の階数'] = df['建物の階数'] #-が入っている部屋の階数のセルを、建物の階数で更新する（'-'は完全一致で検索）
# '-'に続く数字を抽出する関数
def extract_hyphen(text):
    if isinstance(text, str) and '-' in text:
        return pd.Series(text).str.extract(r'-(\d+)',expand=False)[0]
    else:
        return text
df['部屋の階数'] = df['部屋の階数'].astype(str).apply(extract_hyphen) #関数を適用して抽出
#B1を-1に変換する
df['部屋の階数'] = df['部屋の階数'].astype(str).str.replace('B1', '-1')  #築年数のカラムから「新」を削除し、0を投入
df['部屋の階数'] = df['部屋の階数'].astype(str).str.replace('B3', '-3')  #築年数のカラムから「新」を削除し、0を投入
df['部屋の階数'] = df['部屋の階数'].astype(str).str.replace('B4', '-4')  #築年数のカラムから「新」を削除し、0を投入
change_int('部屋の階数') #部屋の階数をint型に修正する


In [56]:
#Step2-5｜面積をfloat型に修正する

#特定のカラムについて指定した文字を削除し、float系に変換する
def delete_text_float(df, column,delete_texts):
    for delete_text_float in delete_texts:
        df[column] = df[column].str.replace(delete_text_float,'') #指定した文字を削除する
        df[column] = df[column].apply(lambda x: float(x) if pd.notna(x) and x.isdigit() else x)
    return df

df['面積'] = df['面積'].replace('', None)  # 空の文字列を'0'に置換
df = delete_text_float(df, '面積','m2') #面積のカラムから「m2」を削除する


In [57]:
#Step2-6｜不要なカラムを削除し、空の列をNoneに変換する
df.drop(columns=['アクセス1', 'アクセス2','アクセス3'], inplace=True) #不要な列を削除する
df = df.replace('', None) #空の文字列をNoneに変換する

In [58]:
#Step2-7｜重複を排除する（所在地、築年数、賃料、間取り、面積、建物の階数）
df.drop_duplicates(subset=['所在地', '築年数','賃料','間取り','面積','建物の階数'], keep='first')
df

,カテゴリ,物件名,所在地,築年数,建物の階数,部屋の階数,賃料,管理費,敷金,礼金,...,バス所要時間1,徒歩所要時間1,路線2,駅名2,バス所要時間2,徒歩所要時間2,路線3,駅名3,バス所要時間3,徒歩所要時間3
0,賃貸アパート,西武新宿線 上石神井駅 2階建 築8年,東京都練馬区上石神井２,8,2,2,185000,4000,0,185000,...,,4,西武新宿線,上井草駅,,19,西武新宿線,武蔵関駅,,20
1,賃貸アパート,西武新宿線 上石神井駅 2階建 築8年,東京都練馬区上石神井２,8,2,2,185000,4000,0,185000,...,,4,西武新宿線,上井草駅,,19,西武新宿線,武蔵関駅,,20
2,賃貸一戸建て,東武伊勢崎線 小菅駅 2階建 築34年,東京都足立区西綾瀬１,34,2,2,148000,0,296000,148000,...,,6,ＪＲ常磐線,綾瀬駅,,10,東京メトロ千代田線,綾瀬駅,,10
3,賃貸マンション,アーブル下落合Ａ棟,東京都新宿区下落合２,12,4,1,100000,4000,100000,100000,...,,7,東京メトロ東西線,高田馬場駅,,7,ＪＲ山手線,目白駅,,7
4,賃貸一戸建て,ＪＲ京浜東北線 王子駅 2階建 築64年,東京都北区栄町,64,2,2,120000,5000,120000,0,...,,10,都電荒川線,栄町駅,,5,東京メトロ南北線,西ケ原駅,,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15766,賃貸一戸建て,都営大江戸線 西新宿五丁目駅 2階建 築56年,東京都新宿区西新宿４,56,2,1,159000,10000,159000,159000,...,10,2,東京メトロ丸ノ内線,西新宿駅,None,11,都営大江戸線,都庁前駅,None,12
15767,賃貸アパート,木原山ロッヂ,東京都大田区山王４,85,3,1,120000,0,120000,0,...,10,10,都営浅草線,馬込駅,None,16,京急本線,大森海岸駅,None,21
15768,賃貸マンション,山王パレス,東京都大田区山王２,57,7,5,180000,0,180000,180000,...,10,2,京急本線,大森海岸駅,None,14,京急本線,平和島駅,None,26
15769,賃貸アパート,グリーンハイム,東京都練馬区関町南３,31,2,1,80000,0,80000,0,...,8,5,西武新宿線,武蔵関駅,None,10,西武新宿線,上石神井駅,None,21


In [59]:
df_columns = df.columns.values
export_csv(df_columns,'df_csv', 'suumo_data_cleansing')

In [4]:
#Step3-1｜スプレッドシート連携に必要なライブラリをインストールする
!pip install gspread
!pip install google-auth
!pip install gspread-dataframe


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [5]:
#Step3-2｜スプレッドシート連携に必要なライブラリをインポートする

import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート

from dotenv import load_dotenv #envファイルを用いた情報を隠す方法を連携
load_dotenv()
import os

In [6]:
#Step3-3｜認証情報関連について設定を行う

# 認証のために機能役割を決めるアクセス先をscopesに設定
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
#.envで設定した変数を取得する
sp_sheet_key = os.getenv('sp_sheet_key')
service_account_file = os.getenv('service_account_file')

# その役割の許可をもらうAPIキーをservice_account_new.jsonから読み込み、credentialsに代入
# 認証キーを使うアクセス先をscopesに代入
credentials = Credentials.from_service_account_file(
    service_account_file, #Google CloudからDLした認証キーのファイルを設定する
    scopes=scopes
)

# 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
gc = gspread.authorize(credentials)

In [7]:
#Step3-4｜Googleスプレッドシートの設定を行う

# 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
# 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
sh = gc.open_by_key(sp_sheet_key)

# 参照するシート名をSP_SHEETに代入
SP_SHEET = 'suumo_data'

# gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
worksheet = sh.worksheet(SP_SHEET)

In [8]:
#Step3-5｜データの読み込みと実行

#csvデータの読み込み
df = pd.read_csv('suumo_data_cleansing.csv') #前回クレンジングを行ったcsvデータを読み込む
df

df_upload = df # スクレイピングで取得した新しいデータdfをdf_newに代入

set_with_dataframe(sh.worksheet(SP_SHEET), df_upload, include_index=False) # set_with_dataframe(スプレッドシート名,対象データ,データフレームで自動生成されるindex数字を含むかどうか)

In [9]:
#Step4-1｜必要なライブラリのインポート
import sqlite3

In [10]:
#Step4-2｜SQL文の記述

#SUUMO.db という名前のDBをなければ作成して接続する
dbname = 'SUUMO.db'

conn = sqlite3.connect(dbname)
cur = conn.cursor()

In [11]:
# tableのnameを"SUUMO.db"とし、読み込んだcsvファイルをsqlに書き込む
# index=False としてindex は書き込まないようにする
df.to_sql('SUUMO.db', conn, if_exists='replace', index=False)

/Users/kento/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
